## Second Version

In [11]:
import time
import cv2
import numpy as np
import os
from insightface import app
from scipy.spatial.distance import cosine
import faiss

In [3]:
# Load the InsightFace model
face_recognition = app.FaceAnalysis()
face_recognition.prepare(ctx_id=0, det_size=(640, 640))

c:\Users\Deciano\.virtualenvs\retinaface-kY4dmbRz\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Deciano/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Deciano/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Deciano/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Deciano/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Deciano/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112,

In [4]:
# Load the face database
database_path = 'Database'  # Replace with the path to your database
face_embeddings = {}
for person_name in os.listdir(database_path):
    person_folder = os.path.join(database_path, person_name)
    if os.path.isdir(person_folder):
        face_embeddings[person_name] = []
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                faces = face_recognition.get(img)
                for face in faces:
                    # Store the normalized embedding of the first face detected
                    face_embeddings[person_name].append(face.normed_embedding)


In [5]:
face_embeddings['Andika'][0].shape

(512,)

In [7]:
embeddings_list = []
names = []

for name, embeddings in face_embeddings.items():
    for embedding in embeddings:
        embeddings_list.append(np.squeeze(embedding))  # Remove the extra dimension (1, 512) -> (512,)
        names.append(name)

known_faces_embeddings = np.vstack(embeddings_list)
known_faces_embeddings.shape

(28, 512)

In [8]:
faiss.normalize_L2(known_faces_embeddings)

index = faiss.IndexFlatL2(512)
index.add(known_faces_embeddings)

In [12]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

fps = 0.0
frame_count = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    faces = face_recognition.get(frame)

    for face in faces:
        bbox = face.bbox.astype(int)
        identity = face.normed_embedding
        identity = identity[np.newaxis, :]
        faiss.normalize_L2(identity)

        distances, indices = index.search(identity, k=1)

        closest_distance = distances[0][0]
        closest_index = indices[0][0]

        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
        cv2.putText(frame, names[closest_index], (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    frame_count += 1
    current_time = time.time()
    elapsed_time = current_time - start_time

    # Update FPS every second
    if elapsed_time > 1.0:
        fps = frame_count / elapsed_time
        start_time = current_time
        frame_count = 0

    # Display FPS on frame
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    cv2.imshow('Face Recognition', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()